# Twitter Account Discovery with CrewAI

This notebook demonstrates how to use [CrewAI](https://github.com/crewAIInc/crewAI) to build a hierarchical agent system that searches Twitter for accounts related to a topic. The search continues until 1000 unique accounts are gathered.

## Install dependencies
Run the following cell if the required packages are not installed.

In [ ]:
!pip install crewai tweepy -q

## Imports and API credentials
Set your API keys as environment variables before running the rest of the notebook:

In [ ]:
import os
TWITTER_CONSUMER_KEY = os.getenv('TWITTER_CONSUMER_KEY')
TWITTER_CONSUMER_SECRET = os.getenv('TWITTER_CONSUMER_SECRET')
TWITTER_ACCESS_TOKEN = os.getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_SECRET = os.getenv('TWITTER_ACCESS_SECRET')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
SERPER_API_KEY = os.getenv('SERPER_API_KEY')


If any of the variables above prints `None`, set them in your environment before continuing.

In [ ]:
print('TWITTER_CONSUMER_KEY:', TWITTER_CONSUMER_KEY)

## Twitter search tool

In [ ]:
import tweepy
from typing import List, Dict

class TwitterSearchTool:
    def __init__(self):
        auth = tweepy.OAuth1UserHandler(
            TWITTER_CONSUMER_KEY,
            TWITTER_CONSUMER_SECRET,
            TWITTER_ACCESS_TOKEN,
            TWITTER_ACCESS_SECRET,
        )
        self.api = tweepy.API(auth)

    def search_accounts(self, query: str, count: int = 20) -> List[Dict]:
        users = self.api.search_users(q=query, count=count)
        return [
            {
                'username': user.screen_name,
                'name': user.name,
                'description': user.description,
                'followers': user.followers_count,
                'profile_url': f'https://twitter.com/{user.screen_name}',
            }
            for user in users
        ]


## Define CrewAI agents

In [ ]:
from crewai import Agent, Task, Crew, Process

searcher = Agent(
    role='Account Finder',
    goal='Identify Twitter accounts related to a given topic',
    backstory='Expert at social media research and data collection',
    tools=[TwitterSearchTool()],
    verbose=True,
)

manager = Agent(
    role='Search Manager',
    goal='Ensure at least 1000 unique accounts are collected',
    backstory='Oversees the search process and removes duplicates',
    verbose=True,
)

search_task = Task(
    description='Search for Twitter accounts related to {topic}. Return results as JSON.',
    expected_output='A JSON list of accounts',
    agent=searcher,
)

manager_task = Task(
    description='Coordinate the search until 1000 unique accounts are gathered.',
    agent=manager,
)

crew = Crew(
    agents=[manager, searcher],
    tasks=[manager_task, search_task],
    process=Process.hierarchical,
)


## Collect accounts

In [ ]:
import pandas as pd

def gather_accounts(topic: str, batch_size: int = 20) -> pd.DataFrame:
    unique = {}
    page = 1
    while len(unique) < 1000:
        print(f'Searching batch {page}...')
        results = searcher.tools[0].search_accounts(query=topic, count=batch_size)
        for account in results:
            unique[account['username']] = account
        print(f'Total unique accounts: {len(unique)}')
        page += 1
    return pd.DataFrame(list(unique.values()))


## Run the search

In [ ]:
topic = 'AI research'
accounts_df = gather_accounts(topic)
accounts_df.head()
